# Reporting Queries

## students' assessments
```sql
CREATE OR ALTER VIEW v_Instructor_Course_Student_Assessment_Scores AS
WITH instructor_students AS (
SELECT cs.instructor_id, cs.program_course_id, se.student_id, COUNT(*) AS number_of_classes    
FROM [LMS].[Institutions].[ClassSchedule] cs 
JOIN Institutions.ClassRooms cr ON cr.classroom_id = cs.classroom_id
JOIN Students.StudentEnrollments se ON se.schedule_id = cs.schedule_id
GROUP BY cs.instructor_id, cs.program_course_id, se.student_id)
SELECT  
	ist.instructor_id, i.instructor_first_name + ' ' + i.instructor_last_name AS instructor_name,
	pc.program_course_id,
	p.program_id,p.program_name, 
	c.course_id, c.course_name,
	s.student_id, s.student_first_name + ' ' + s.student_last_name AS student_name,
	assc.assessment_id, a.assessment_name, assc.student_score, assc.assessment_total_score,
	assc.student_score/assc.assessment_total_score * 100 AS result_percentage
FROM instructor_students ist
JOIN Students.AssessmentScores assc 
	ON assc.student_id = ist.student_id and assc.program_course_id = ist.program_course_id
JOIN Programs.Assessments a
	ON a.assessment_id = assc.assessment_id
JOIN Institutions.Instructors i ON i.instructor_id = ist.instructor_id
JOIN Programs.ProgramCourses pc ON pc.program_course_id = ist.program_course_id
JOIN Institutions.Programs p ON p.program_id = pc.program_id
JOIN Programs.Courses c ON c.course_id = pc.course_id
JOIN Students.Students s ON s.student_id = ist.student_id
```

## Exams
```sql
WITH exams_subs AS (
    SELECT     
        c.course_name, c.course_id, s.student_id, 
        e.exam_name, e.exam_id, q.question_text, 
        t.topic_name, 
        a.answer_text, a.right_answer, s.student_institution_program_id
    FROM  
        Students.ExamSubmissions AS es 
    INNER JOIN
        Students.Students AS s ON s.student_id = es.student_id 
    INNER JOIN
        Programs.Exams AS e ON e.exam_id = es.exam_id 
    INNER JOIN
        Programs.Questions AS q ON q.question_id = es.question_id 
    INNER JOIN
        Programs.Answers AS a ON a.answer_id = es.answer_id 
    INNER JOIN
        Programs.Courses AS c ON c.course_id = e.course_id 
    INNER JOIN
        Programs.Topics AS t ON t.topic_id = q.topic_id), 
enrolled_program_courses AS(
    SELECT DISTINCT 
        se.student_id, cs.instructor_id, 
        pc.program_course_id, pc.course_id, pc.program_id
    FROM 
        Students.StudentEnrollments AS se 
    INNER JOIN
        Institutions.ClassSchedule AS cs ON cs.schedule_id = se.schedule_id 
    INNER JOIN
        Programs.ProgramCourses AS pc ON cs.program_course_id = pc.program_course_id)
    SELECT 
        p.program_name, es.course_name, es.course_id, 
        epc.instructor_id, es.student_id, 
        es.exam_name, es.question_text, es.topic_name, 
        es.answer_text, es.right_answer * 1 AS right_answer_encoded, 
        es.right_answer, i.institution_name, es.exam_id
    FROM 
        exams_subs AS es 
    INNER JOIN
        enrolled_program_courses AS epc ON es.student_id = epc.student_id AND es.course_id = epc.course_id 
    INNER JOIN
        Institutions.Programs AS p ON p.program_id = epc.program_id 
    INNER JOIN
        Institutions.InstitutionPrograms AS ip_ ON 
            ip_.institution_program_id = es.student_institution_program_id 
    INNER JOIN
        Institutions.Institutions AS i ON i.institution_id = ip_.institution_id
```

Procedure
```sql
CREATE OR ALTER PROCEDURE sp_GetAssessmentScores
    @instructor_id INT = NULL,
    @student_id INT = NULL
AS 
BEGIN 
    -- Validation to check if the provided instructor_id or student_id is available
    IF (@instructor_id IS NOT NULL AND @instructor_id NOT IN (SELECT DISTINCT instructor_id FROM v_Instructor_Course_Student_Assessment_Scores)) 
    BEGIN
        PRINT 'The provided instructor_id  "'+ CAST(@instructor_id AS VARCHAR) +'"  is not available. Please try again with a valid ID.';
        RETURN;
    END;
	IF (@student_id IS NOT NULL AND @student_id NOT IN (SELECT DISTINCT student_id FROM v_Instructor_Course_Student_Assessment_Scores))
	    BEGIN
        PRINT 'The provided student_id "'+ CAST(@student_id AS VARCHAR) +'" is not available. Please try again with a valid ID';
        RETURN;
    END;
    -- Validation to check if only student_id is provided without instructor_id
    IF (@instructor_id IS NULL AND @student_id IS NOT NULL)
    BEGIN
        PRINT 'Please provide a valid instructor_id (the instructor must teach the student)';
        RETURN;
    END;

    -- Retrieve all records when both instructor_id and student_id are NULL
    IF (@instructor_id IS NULL AND @student_id IS NULL)
    BEGIN
        PRINT '@instructor_id AND @student_id are NULL';
        SELECT * 
        FROM [dbo].[v_Instructor_Course_Student_Assessment_Scores]
        ORDER BY instructor_id, program_course_id, student_id;
        RETURN;
    END;

    -- Retrieve records based on student_id when only instructor_id is NULL
    IF (@instructor_id IS NOT NULL AND @student_id IS NULL)
    BEGIN
        PRINT '@student_id is NULL';
        SELECT * FROM [dbo].[v_Instructor_Course_Student_Assessment_Scores]
        WHERE instructor_id = @instructor_id;
        RETURN;
    END;

    -- Retrieve records based on both instructor_id and student_id
    IF (@instructor_id IS NOT NULL AND @student_id IS NOT NULL)
    BEGIN
        PRINT 'instructor_id: '+ CAST(@instructor_id AS VARCHAR)+  ', student_id:' + CAST(@student_id AS VARCHAR);
        SELECT * FROM [dbo].[v_Instructor_Course_Student_Assessment_Scores]
        WHERE instructor_id = @instructor_id AND student_id = @student_id;
        RETURN;
    END;

    PRINT 'ERROR: No condition satisfied. Try again.';
END;
```

## Schedule 
time until next class, current class and up next class
```sql
CREATE OR ALTER VIEW v_schedule AS
WITH schedule_view AS (
SELECT 
	I.institution_name, P.program_name, C.course_name, C.course_year_number,
	CS.instructor_id,
	TS.day_of_week, CAST(TS.time_slot_start_time AS TIME) AS time_slot_start_time, TS.time_slot_end_time, CS.schedule_id,
	FORMAT(GETDATE(), 'dddd') AS current_day_of_week,
    FORMAT(GETDATE(), 'HH:mm:ss') AS time_now,
	FORMAT(GETDATE(), 'dd-MM-yyyy') AS date_today,
	DATEPART(WEEKDAY, GETDATE()) AS today_week_day_encoding,
	(CONVERT(DATETIME, FORMAT(GETDATE(), 'dd-MM-yyyy'),103) + 
					CONVERT(DATETIME, time_slot_start_time, 108)) AS time_slot_datetime,
    CASE 
        WHEN TS.day_of_week = 'Saturday' THEN 7
        WHEN TS.day_of_week = 'Sunday' THEN 1
        WHEN TS.day_of_week = 'Monday' THEN 2
        WHEN TS.day_of_week = 'Tuesday' THEN 3
        WHEN TS.day_of_week = 'Wednesday' THEN 4
        WHEN TS.day_of_week = 'Thursday' THEN 5
        WHEN TS.day_of_week = 'Friday' THEN 6
    END AS day_of_week_encoding_sunday,
	CR.classroom_name, SE.student_id
FROM Institutions.ClassSchedule CS
JOIN Institutions.ClassRooms CR ON CS.classroom_id = CR.classroom_id
JOIN Students.StudentEnrollments SE ON SE.schedule_id = CS.schedule_id
JOIN Programs.ProgramCourses PC ON PC.program_course_id = CS.program_course_id
JOIN Institutions.InstitutionPrograms IP ON IP.program_id = PC.program_id
JOIN Institutions.Institutions I ON I.institution_id = IP.institution_id
JOIN Programs.Courses C ON C.course_id = PC.course_id
JOIN Institutions.Programs P ON P.program_id = PC.program_id
JOIN Institutions.TimeSlots TS ON TS.time_slot_id = CS.time_slot_id
),
time_and_status AS (SELECT 
		institution_name, program_name, course_name, course_year_number,
		instructor_id, student_id, classroom_name,
		schedule_id, day_of_week,day_of_week_encoding_sunday, 
		current_day_of_week,today_week_day_encoding,  
		time_slot_start_time, time_slot_end_time, time_now, time_slot_datetime,
		CASE 
			WHEN today_week_day_encoding = day_of_week_encoding_sunday
				 AND FORMAT(GETDATE(),'hh:mm:ss') < time_slot_start_time
				 THEN 0
			WHEN today_week_day_encoding = day_of_week_encoding_sunday
				 AND FORMAT(GETDATE(),'hh:mm:ss') >= time_slot_start_time
				 THEN 6
			WHEN today_week_day_encoding < day_of_week_encoding_sunday
				THEN day_of_week_encoding_sunday - today_week_day_encoding -1
			WHEN today_week_day_encoding > day_of_week_encoding_sunday 
				THEN ABS(today_week_day_encoding - 7) + day_of_week_encoding_sunday -1
			ELSE NULL
		END AS days_till_class,
		CASE 
			WHEN CAST(DATEPART(HOUR,time_slot_start_time) AS INT) = CAST (DATEPART(HOUR,GETDATE()) AS INT)
				 THEN 0
			WHEN CAST(DATEPART(HOUR,time_slot_start_time) AS INT) > CAST (DATEPART(HOUR,GETDATE()) AS INT)
				THEN CAST(DATEPART(HOUR,time_slot_start_time) AS INT) - CAST (DATEPART(HOUR,GETDATE()) AS INT) -1
			WHEN CAST(DATEPART(HOUR,time_slot_start_time) AS INT) < CAST (DATEPART(HOUR,GETDATE()) AS INT)
				THEN (24 - CAST (DATEPART(HOUR,GETDATE()) AS INT)) + CAST(DATEPART(HOUR,time_slot_start_time) AS INT)-1
			ELSE NULL
		END AS hours_till_class,
		CASE 
			WHEN CAST(DATEPART(MINUTE,time_slot_start_time) AS INT) = CAST (DATEPART(MINUTE,GETDATE()) AS INT)
				 THEN 0
			WHEN CAST(DATEPART(MINUTE,time_slot_start_time) AS INT) > CAST (DATEPART(MINUTE,GETDATE()) AS INT)
				THEN CAST(DATEPART(MINUTE,time_slot_start_time) AS INT) - CAST (DATEPART(MINUTE,GETDATE()) AS INT) -1
			WHEN CAST(DATEPART(MINUTE,time_slot_start_time) AS INT) < CAST (DATEPART(MINUTE,GETDATE()) AS INT)
				THEN (60 - CAST (DATEPART(MINUTE,GETDATE()) AS INT)) + CAST(DATEPART(MINUTE,time_slot_start_time) AS INT)-1
			ELSE NULL
		END AS minutes_till_class,
		CASE 
			WHEN CAST(DATEPART(SECOND,time_slot_start_time) AS INT) = CAST (DATEPART(SECOND,GETDATE()) AS INT)
				 THEN 0
			WHEN CAST(DATEPART(SECOND,time_slot_start_time) AS INT) > CAST (DATEPART(SECOND,GETDATE()) AS INT)
				THEN CAST(DATEPART(SECOND,time_slot_start_time) AS INT) - CAST (DATEPART(SECOND,GETDATE()) AS INT) -1
			WHEN CAST(DATEPART(SECOND,time_slot_start_time) AS INT) < CAST (DATEPART(SECOND,GETDATE()) AS INT)
				THEN (60 - CAST (DATEPART(SECOND,GETDATE()) AS INT)) + CAST(DATEPART(SECOND,time_slot_start_time) AS INT)-1
			ELSE NULL
		END AS seconds_till_class,
			CASE 
		WHEN 
			FORMAT(GETDATE(), 'dddd') = day_of_week 
			AND FORMAT(GETDATE(), 'HH:mm:ss') 
				BETWEEN time_slot_start_time AND time_slot_end_time THEN 1
		WHEN 
			FORMAT(GETDATE(), 'dddd') = day_of_week 
			AND FORMAT(GETDATE(), 'HH:mm:ss') 
				BETWEEN DATEADD(HOUR, 1.5, time_slot_start_time)  AND DATEADD(HOUR, 1.5, time_slot_end_time) THEN 2
		ELSE 0
	END AS inprogress_upnext
FROM schedule_view
)
select *,
		CAST(days_till_class AS VARCHAR) +' days '+
		+ CAST(hours_till_class AS VARCHAR) +' hours '+
		+ CAST(minutes_till_class AS VARCHAR) +' minutes '+
		+ CAST(seconds_till_class AS VARCHAR) +' Seconds' as time_till_class
from time_and_status
```

# Not Used

### Classroom Time Table 

#### TimeTable View 
```sql
CREATE OR ALTER VIEW v_ClassTimeTable AS
	SELECT 
            ts.day_of_week,
            cr.classroom_name,
            STRING_AGG(CASE
                WHEN ts.time_slot_start_time = '08:00:00.0000000' 
                    THEN 
                        c.course_name + ' || Class: ' + cr.classroom_name + ' || Instructor: ' + i.instructor_first_name + ' ' + i.instructor_last_name
                ELSE NULL
            END, '') AS "08:00:00",
            STRING_AGG(CASE
                WHEN ts.time_slot_start_time = '09:30:00.0000000' 
                    THEN 
                        c.course_name + ' || Class: ' + cr.classroom_name + ' || Instructor: ' + i.instructor_first_name + ' ' + i.instructor_last_name
                ELSE NULL
            END, '') AS "09:30:00",
            STRING_AGG(CASE
                WHEN ts.time_slot_start_time = '11:00:00.0000000' 
                    THEN 
                        c.course_name + ' || Class: ' + cr.classroom_name + ' || Instructor: ' + i.instructor_first_name + ' ' + i.instructor_last_name
                ELSE NULL
            END, '') AS "11:00:00",
            STRING_AGG(CASE
                WHEN ts.time_slot_start_time = '12:30:00.0000000' 
                    THEN 
                        c.course_name + ' || Class: ' + cr.classroom_name + ' || Instructor: ' + i.instructor_first_name + ' ' + i.instructor_last_name
                ELSE NULL
            END, '') AS "12:30:00",
            STRING_AGG(CASE
                WHEN ts.time_slot_start_time = '14:00:00.0000000' 
                    THEN 
                        c.course_name + ' || Class: ' + cr.classroom_name + ' || Instructor: ' + i.instructor_first_name + ' ' + i.instructor_last_name
                ELSE NULL
            END, '') AS "14:00:00"
        FROM [LMS].[Institutions].[ClassSchedule] cs 
        JOIN Institutions.ClassRooms cr ON cr.classroom_id = cs.classroom_id
        JOIN Institutions.TimeSlots ts ON ts.time_slot_id = cs.time_slot_id
        JOIN Institutions.Instructors i ON i.instructor_id = cs.instructor_id
        JOIN Programs.ProgramCourses pc ON pc.program_course_id = cs.program_course_id
        JOIN Institutions.Programs p ON p.program_id = pc.program_id
        JOIN Programs.Courses c ON c.course_id = pc.course_id
   
        GROUP BY cr.classroom_name, ts.day_of_week 
```

#### Filter Proc
```sql
CREATE OR ALTER PROCEDURE sp_ClassTimeTable 
    @classroom_name NVARCHAR(255) = NULL
AS
BEGIN
    -- Parameter Validation: Check if @classroom_name is not null
    IF @classroom_name IS NULL
    BEGIN
        PRINT 'Warning: @classroom_name parameter is Null. Please provide a value.' + CHAR(13) + CHAR(10);
		SELECT * FROM v_ClassTimeTable;
        RETURN;
    END;

    -- Check if the classroom exists
    IF EXISTS (SELECT 1 FROM [Institutions].[ClassRooms] WHERE classroom_name = @classroom_name)
    BEGIN
        SELECT * FROM v_ClassTimeTable
		WHERE classroom_name = @classroom_name
		ORDER BY classroom_name;
    END
    ELSE
    BEGIN
        PRINT 'Error: Classroom "' + @classroom_name + '" does not exist or the classroom name is wrong.' + CHAR(13) + CHAR(10);
    END;
END;
```

```SQL
EXEC sp_ClassTimeTable 
    @classroom_name = '1A';
```

### Instructor Time Table
```sql
CREATE OR ALTER VIEW v_InstructorTimeTable AS
SELECT 
	ts.day_of_week,
	i.instructor_id,
	STRING_AGG(CASE
		WHEN ts.time_slot_start_time = '08:00:00.0000000' 
			THEN 
				c.course_name 
				+ ' || Class: ' + cr.classroom_name 
				+ ' || Instructor: ' +
				i.instructor_first_name +  ' ' + i.instructor_last_name
		ELSE NULL
	END,'') AS "08:00:00",
	STRING_AGG(CASE
		WHEN ts.time_slot_start_time = '09:30:00.0000000' 
			THEN 
				c.course_name 
				+ ' || Class: ' + cr.classroom_name 
				+ ' || Instructor: ' +
				i.instructor_first_name +  ' ' + i.instructor_last_name
		ELSE NULL
	END,'') AS "09:30:00",
	STRING_AGG(CASE
		WHEN ts.time_slot_start_time = '11:00:00.0000000' 
			THEN
				c.course_name 
				+ ' || Class: ' + cr.classroom_name 
				+ ' || Instructor: ' +
				i.instructor_first_name +  ' ' + i.instructor_last_name
		ELSE NULL
	END,'') AS "11:00:00",
	STRING_AGG(CASE
		WHEN ts.time_slot_start_time = '12:30:00.0000000' 
			THEN 
				c.course_name 
				+ ' || Class: ' + cr.classroom_name 
				+ ' || Instructor: ' +
				i.instructor_first_name +  ' ' + i.instructor_last_name
		ELSE NULL
	END,'') AS "12:30:00",
	STRING_AGG(CASE
		WHEN ts.time_slot_start_time = '14:00:00.0000000' 
			THEN 
				c.course_name 
				+ ' || Class: ' + cr.classroom_name 
				+ ' || Instructor: ' +
				i.instructor_first_name +  ' ' + i.instructor_last_name
		ELSE NULL
	END,'') AS "14:00:00"
FROM [LMS].[Institutions].[ClassSchedule] cs 
JOIN Institutions.ClassRooms cr ON cr.classroom_id = cs.classroom_id
JOIN Institutions.TimeSlots ts ON ts.time_slot_id = cs.time_slot_id
JOIN Institutions.Instructors i ON i.instructor_id = cs.instructor_id
JOIN Programs.ProgramCourses pc ON pc.program_course_id = cs.program_course_id
JOIN Institutions.Programs p ON p.program_id = pc.program_id
JOIN Programs.Courses c ON c.course_id = pc.course_id
GROUP BY i.instructor_id, ts.day_of_week 
```


```Sql
CREATE OR ALTER PROCEDURE sp_InstructorTimeTable 
	@instructor_id INT = NULL
AS
BEGIN
	-- Parameter Validation (Example: Ensure @instructor_id is not null)
    IF @instructor_id IS NULL
    BEGIN
        PRINT 'Warning @instructor_id parameter is Null Please provide a value';
		SELECT * 
		FROM v_InstructorTimeTable
		ORDER BY instructor_id;
        RETURN;
    END;

	IF @instructor_id IN (SELECT DISTINCT instructor_id from [Institutions].[ClassSchedule])
	BEGIN
		SELECT * 
		FROM v_InstructorTimeTable
		WHERE instructor_id = @instructor_id
		ORDER BY instructor_id;
	END
	ELSE
	BEGIN
		PRINT 'Instructor ID "' + CAST(@instructor_id AS VARCHAR) + '" has not been assigned to a class or instructor id is wrong'
	END;
END;
```

```SQL
EXEC sp_InstructorTimeTable 
	@instructor_id = 5;
```

### Student Time Table
#### Table View
```sql

CREATE OR ALTER VIEW v_StudentTimeTable AS 
	SELECT 
        ts.day_of_week,
        se.student_id,
        STRING_AGG(CASE
            WHEN ts.time_slot_start_time = '08:00:00.0000000' 
                THEN 
                    c.course_name + ' | ' 
                    + cr.classroom_name + ' | ' 
                    + i.instructor_first_name +  ' ' 
                    + i.instructor_last_name
            ELSE NULL
        END, '') AS "08:00:00",
        STRING_AGG(CASE
            WHEN ts.time_slot_start_time = '09:30:00.0000000' 
                THEN  
                    c.course_name + ' | ' 
                    + cr.classroom_name + ' | ' 
                    + i.instructor_first_name +  ' ' 
                    + i.instructor_last_name
            ELSE NULL
        END, '') AS "09:30:00",
        STRING_AGG(CASE
            WHEN ts.time_slot_start_time = '11:00:00.0000000' 
                THEN 
                    c.course_name + ' | ' 
                    + cr.classroom_name + ' | ' 
                    + i.instructor_first_name +  ' ' 
                    + i.instructor_last_name
            ELSE NULL
        END, '') AS "11:00:00",
        STRING_AGG(CASE
            WHEN ts.time_slot_start_time = '12:30:00.0000000' 
                THEN  
                    c.course_name + ' | ' 
                    + cr.classroom_name + ' | ' 
                    + i.instructor_first_name +  ' ' 
                    + i.instructor_last_name
            ELSE NULL
        END, '') AS "12:30:00",
        STRING_AGG(CASE
            WHEN ts.time_slot_start_time = '14:00:00.0000000' 
                THEN 
                    c.course_name + ' || Class: ' 
                    + cr.classroom_name + ' || Instructor: ' 
                    + i.instructor_first_name +  ' ' 
                    + i.instructor_last_name
            ELSE NULL
        END, '') AS "14:00:00"
    FROM [LMS].[Institutions].[ClassSchedule] cs 
    JOIN Institutions.ClassRooms cr ON cr.classroom_id = cs.classroom_id
    JOIN Institutions.TimeSlots ts ON ts.time_slot_id = cs.time_slot_id
    JOIN Institutions.Instructors i ON i.instructor_id = cs.instructor_id
    JOIN Programs.ProgramCourses pc ON pc.program_course_id = cs.program_course_id
    JOIN Institutions.Programs p ON p.program_id = pc.program_id
    JOIN Programs.Courses c ON c.course_id = pc.course_id
    JOIN Students.StudentEnrollments se ON se.schedule_id = cs.schedule_id
    GROUP BY se.student_id, ts.day_of_week, i.instructor_id 
```



#### Filter Procedure
```sql
CREATE OR ALTER PROCEDURE sp_StudentTimeTable 
    @student_id INT = NULL
AS
BEGIN
    -- Parameter Validation: Check if @student_id is not null
    IF @student_id IS NULL
    BEGIN
        PRINT 'WARNING: @student_id parameter is Null. Please provide a value.' + CHAR(13) + CHAR(10);
		SELECT  * FROM v_StudentTimeTable;
        RETURN;
    END;

    -- Check if the student is enrolled
    IF EXISTS (SELECT 1 FROM [Students].[StudentEnrollments] WHERE student_id = @student_id)
    BEGIN
        SELECT  * FROM v_StudentTimeTable
        WHERE student_id = @student_id
        ORDER BY student_id;
    END
    ELSE
    BEGIN
        PRINT 'WARNING: Student id "' + CAST(@student_id AS VARCHAR)  + '" is not enrolled or the student id is wrong.' + CHAR(13) + CHAR(10);
    END;
END;

```

```SQL
EXEC sp_StudentTimeTable
	@student_id = 8;
```